# Data Preparation

## Import Libraries

In [1]:
# Import the Libraries for Google Drive Document Retrieval
from pydrive.auth import GoogleAuth
import pandas as pd
from pydrive.drive import GoogleDrive
import glob

#Libraries to handel and prepare data 
import importlib
import datetime
import holidays
import progressbar
from time import sleep
import pycountry
import math

## Authenticate to Google Drive 
(Request access to lucasbotellaroca@gmail.com if not conceded)
Retrieve all files from different folders

In [2]:
# A browser window will open. login using the appropriate account.
gauth = GoogleAuth()
gauth.LocalWebserverAuth() #

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=267819126046-3da5rmgml4iemor49ermcv23tnaj02t5.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


In [3]:
drive = GoogleDrive(gauth)

In [4]:
drive

### Holidays auxiliary code
Evaluates wether a given date for a given country is a national holiday or not.

In [5]:
def getHoliday(country, date):
    
    res=False
    if hasattr(holidays, country):
        function_string = 'holidays.' + country
        mod_name, func_name = function_string.rsplit('.',1)
        mod = importlib.import_module(mod_name)
        func = getattr(mod, func_name)
        if(isinstance(func, int)==False):
            result = func()
            res = result.get(date) is not None
 
    return res

## Retrieve data from Google Drive

### Read single file 
Reads a csv file in a certain drive folder, skipping n first nrows. 
Returns a dataframe, if not found returns empty dataframe

In [6]:
def read_csv_file(folder, fileName, nrows):
    
    listed_mortality = drive.ListFile({'q': "title contains '" + fileName + ".csv' and '" + folder + "' in parents"}).GetList()
    
    if len(listed_mortality)>0:
        title = listed_mortality[0]['title']
        id = listed_mortality[0]['id']
        each_file = drive.CreateFile({'id': id})
        each_file.GetContentFile(title)
        df = pd.read_csv(title, skiprows=nrows)
    else:
        
        df=pd.DataFrame()
    
    return df

### Grouping Policies and Trends

##### group_policies()
Method used to group policies as the average of policies in the selected week. It has been decided to group values on Sundays, since excess mortality data comes out every Sunday. The rest of the variables are calculated as detailed below:

Day x Policies = policies and trends of x - 18 days.
Week x Policies = mean of policies and trends in the selected week.
Every day in the week has a policies related from 18 previous days (average death limit for covid death in patients)

In [7]:
def group_policies(filename):

    df = pd.read_csv(filename)
    df['Holiday']=df.apply(lambda x: getHoliday(x['Code'], x['Day']), axis=1)
    df['Date'] = pd.to_datetime(df['Day']) + pd.to_timedelta(18, unit='d')
    
    #Group to closest Sunday
    df = df.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
        
    return df

##### df_group_policies_trends_others()
Loops through all files contained in folder and aggregate them using "group_policies" auxiliary method as detailed above.

In [8]:
def df_group_policies_trends_others():
    listed = drive.ListFile({'q': "title contains '.csv' and '1XpkEI-8Zfe0IQCzoa9Pas6Y_hpjGVhGE' in parents"}).GetList()
    list_id = []
    list_title = []
    for file in listed:
        list_id.append(file['id'])
        list_title.append(file['title'])
        
    bar = progressbar.ProgressBar(maxval=15, \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    df = pd.DataFrame()

    i=0

    for id, title in zip(list_id, list_title):
        print(title)
        bar.update(i+1)
        sleep(0.1)
        
        each_file = drive.CreateFile({'id': id})
        each_file.GetContentFile(title)

        if(i==0):
            df = group_policies(title)
        else:
            df_aux = group_policies(title)
            df = pd.merge(df, df_aux, on=["Code", "Date"])
        i=i+1
            
    return df

## Create Dataframe with all Features

### Policies, Mobility Trends and Holidays

The call to this method reads and groups data as described in df_group_policies_trends_others and takes 6 minutes

In [9]:
%%time
df_policies=df_group_policies_trends_others()

covid-contact-tracing.csv


changes-visitors-covid.csv


debt-relief-covid.csv


income-support-covid.csv


covid-19-testing-policy.csv


...

stay-at-home-covid.csv


public-gathering-rules-covid.csv


public-events-covid.csv


workplace-closures-covid.csv


school-closures-covid .csv
CPU times: user 6min 31s, sys: 2.99 s, total: 6min 34s
Wall time: 7min 26s


In [79]:
#Delete duplicated Holiday columns and view output
del df_policies['Holiday_x']
del df_policies['Holiday_y']
df_policies['Holiday']=df_policies['Holiday']*7
df_policies

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.000000,0.285714,0.0,0.0,0.0,0.0,0.0,1.285714,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,0.0,1.000000,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,0.0,1.000000,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,0.0,1.000000,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,0.0,0.571429,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0


In [80]:
#Lets check for Holiday in Spain related to previous 18 days.
df_policies[(df_policies['Code']=='ESP') & (df_policies['Holiday'] > 0)]

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,restrictions_internal_movements,close_public_transport,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday
2189,ESP,2020-05-24,1.0,-84.897857,-46.714286,26.448857,-73.347000,-58.612286,-63.204000,2.0,...,1.0,1.0,2.0,0.857143,2.0,4.0,2.0,3.0,3.0,1.0
2204,ESP,2020-09-06,1.0,-16.795857,-4.857000,3.489571,-35.918429,66.857143,-31.877571,2.0,...,2.0,0.0,2.0,3.000000,1.0,4.0,2.0,2.0,2.0,1.0
2212,ESP,2020-11-01,1.0,-24.816286,-3.571571,5.204000,-24.102143,6.000000,-20.979714,2.0,...,2.0,0.0,2.0,3.285714,1.0,4.0,2.0,2.0,3.0,1.0
2215,ESP,2020-11-22,1.0,-33.673571,-1.632714,8.020286,-27.571429,-7.714286,-19.734714,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,1.0,1.0
2220,ESP,2020-12-27,1.0,-32.979429,-0.265286,9.469429,-29.898000,-19.550857,-22.979714,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,1.0,2.0
2223,ESP,2021-01-17,1.0,-28.591857,2.510143,10.469429,-34.102143,-12.571429,-37.040857,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,3.0,1.0
2224,ESP,2021-01-24,1.0,-31.203857,-1.571571,12.693857,-40.061143,-16.612143,-43.591857,2.0,...,2.0,0.0,2.0,4.000000,2.0,4.0,2.0,2.0,3.0,2.0


### Weather Data

Collects weather data from all stations in the world and relate them to each country taking average of values. The treatment and creation of this file can be found in the file weather_data which uses Google BigQuery and Google Maps API in order to retrieve all the desired information

In [13]:
df_weather=read_csv_file('1yXs7YOhttKS7STixDmqD1kv1woRYWfUd','weather_final_df',0)
df_weather=df_weather.dropna()
#Cast country code to standard used in our dataset
for country in pycountry.countries: 
    df_weather.loc[df_weather['country']==country.alpha_2, 'Code'] = country.alpha_3
#Apply grouping
df_weather=df_weather[['Code','date', 'temp', 'prcp']]
df_weather['Date'] = pd.to_datetime(df_weather['date']) + pd.to_timedelta(18, unit='d')
df_weather = df_weather.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()

,Code,Date,temp,prcp
0,AFG,2020-01-19,3.178571,0.037143
1,AFG,2020-01-26,2.465465,0.309009
2,AFG,2020-02-02,-0.958463,0.100000
3,AFG,2020-02-09,-0.615304,0.082075
4,AFG,2020-02-16,1.666667,0.031604
...,...,...,...,...
11962,ZWE,2021-03-21,23.790598,0.082308
11963,ZWE,2021-03-28,24.935185,0.000333
11964,ZWE,2021-04-04,24.713992,0.002963
11965,ZWE,2021-04-11,23.766667,0.000400


In [81]:
df_weather

,Code,Date,temp,prcp
0,AFG,2020-01-19,3.178571,0.037143
1,AFG,2020-01-26,2.465465,0.309009
2,AFG,2020-02-02,-0.958463,0.100000
3,AFG,2020-02-09,-0.615304,0.082075
4,AFG,2020-02-16,1.666667,0.031604
...,...,...,...,...
11962,ZWE,2021-03-21,23.790598,0.082308
11963,ZWE,2021-03-28,24.935185,0.000333
11964,ZWE,2021-04-04,24.713992,0.002963
11965,ZWE,2021-04-11,23.766667,0.000400


In [82]:
df_merge=df_policies

In [83]:
#Combine original dataframe with retrieved information
df_merge = pd.merge(df_merge, df_weather, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,public_information_campaigns,facial_coverings,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,1.0,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.285714,0.0,0.0,0.0,0.0,0.0,1.285714,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,23.986111,0.0
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.000000,3.0,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN


### Doctors and Nurses

Doctors and nurses information per 1000 

In [84]:
doctors_df=read_csv_file('17wrWTb6BZMqWkNrqYipTQXo7IOxqekOf','doctors',0)
nurses_df=read_csv_file('17wrWTb6BZMqWkNrqYipTQXo7IOxqekOf','nurses',0)

doctors_df=doctors_df[['LOCATION', 'TIME', 'Value']]
doctors_df_18=doctors_df[doctors_df['TIME'] == 2018]
nurses_df=nurses_df[['LOCATION', 'TIME', 'Value']]
nurses_df=nurses_df[nurses_df['TIME'] == 2018]

for index, row in doctors_df.iterrows(): 
    df_merge.loc[df_merge['Code']==row.LOCATION , 'doctors_per_1000'] = row.Value
for index, row in nurses_df.iterrows(): 
    df_merge.loc[df_merge['Code']==row.LOCATION , 'nurses_per_1000'] = row.Value
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,stay_home_requirements,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.0,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.0,0.0,0.0,1.285714,1.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.0,3.0,2.0,2.0,1.000000,0.0,23.986111,0.0,NaN,NaN
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.0,3.0,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN


### Hospital Beds

Hospital beds per 1000 

In [85]:
hospital_beds_df=read_csv_file('1YlfutCzBqYwPhZw-7OsfLC3k1uv1LYfa','hospital_beds', 3)
hospital_beds_df[hospital_beds_df['Country Code']=='ESP']
#Select latest value between 2010 and 2020 
years=hospital_beds_df.columns[54:-1]

latest_values=[]

#Get latest value for hospital beds
for index, row in hospital_beds_df.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])

hospital_beds_df['latest_value']=latest_values
hospital_beds_df=hospital_beds_df[['Country Code', 'latest_value']]
hospital_beds_df.columns=['Code', 'beds_per_1000']
hospital_beds_df.head()

,Code,beds_per_1000
0,ABW,NaN
1,AFG,0.39
2,AGO,NaN
3,ALB,2.89
4,AND,NaN


In [86]:
df_merge = pd.merge(df_merge, hospital_beds_df, on=["Code"], how="left")
df_merge.head()

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,restriction_gatherings,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.0,0.0,1.285714,1.0,NaN,NaN,NaN,NaN,NaN


### International Tourism

Collect international toruism data from previous years and select last occurence 

In [87]:
tourism_df=read_csv_file('1L_YnKR2utwlvzE2ykY47xGsZeLwi3RKu','international-tourism-number-of-arrivals', 0)

tourism_df=tourism_df.dropna()
years=tourism_df['Year'].unique()
countries=tourism_df['Code'].unique()

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

max_year=1995

for country in countries:
    max_year=1995
    for year in years:
        if((year > max_year) & (len(tourism_df[(tourism_df['Year']==year) & (tourism_df['Code']==country)]) > 0)):
            max_year = year

    latest_values.append(tourism_df[(tourism_df['Year']==max_year) & (tourism_df['Code']==country)]['International tourism, number of arrivals'].values[0])

tourism_df_latest=pd.DataFrame()
tourism_df_latest['Code']=countries
tourism_df_latest['number_of_arrivals']=latest_values
tourism_df_latest  

,Code,number_of_arrivals
0,ALB,4.070000e+06
1,DZA,2.039000e+06
2,ASM,2.010000e+04
3,AND,2.831000e+06
4,AGO,3.970000e+05
...,...,...
198,VNM,1.001300e+07
199,OWID_WRL,1.244961e+09
200,YEM,3.667000e+05
201,ZMB,9.560000e+05


In [88]:
df_merge = pd.merge(df_merge, tourism_df_latest, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,cancel_public_events,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,0.0,1.285714,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.0,2.0,1.000000,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.0,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0


### Urbanization

Collects % urbanization population data in each country from previous years and select last occurence  

In [89]:
urbanization_df=read_csv_file('1XrFzNCmJRUYxATxI6gSfwXrk9aXjYP2K','share-of-population-urban', 0)

urbanization_df=urbanization_df.dropna()
years=urbanization_df['Year'].unique()
countries=urbanization_df['Code'].unique()

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for country in countries:
    max_year=2010
    for year in years:
        if((year > max_year) & (len(urbanization_df[(urbanization_df['Year']==year) & (urbanization_df['Code']==country)]) > 0)):
            max_year = year

    latest_values.append(urbanization_df[(urbanization_df['Year']==max_year) & (urbanization_df['Code']==country)]['Urban population (% of total)'].values[0])

urbanization_df_latest=pd.DataFrame()
urbanization_df_latest['Code']=countries
urbanization_df_latest['urban_population']=latest_values
urbanization_df_latest

,Code,urban_population
0,AFG,25.250000
1,ALB,59.383000
2,DZA,72.052000
3,ASM,87.170000
4,AND,88.150000
...,...,...
210,VNM,35.213000
211,OWID_WRL,54.826518
212,YEM,36.016000
213,ZMB,42.976000


In [90]:
df_merge = pd.merge(df_merge, urbanization_df_latest, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,workplace_closures,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.0,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,0.0,1.285714,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,2.0,1.000000,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,2.0,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237


### Vaccination

Retrieve vaccination data grouping done by weekly on Sundays  

In [91]:
vaccination_df=read_csv_file('1mLYB7CX0BYOkGqRp6QWhDKV_FrXOlDIr','covid-vaccination-doses-per-capita', 0)
vaccination_df['Date'] = pd.to_datetime(vaccination_df['Day']) + pd.to_timedelta(18, unit='d')
vaccination_df=vaccination_df[['Code', 'Date', 'total_vaccinations_per_hundred']]
vaccination_df = vaccination_df.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
vaccination_df.columns=['Code', 'Date', 'total_vaccinations_per_100']
vaccination_df

,Code,Date,total_vaccinations_per_100
0,ABW,2021-04-18,24.130000
1,ABW,2021-05-02,41.670000
2,ABW,2021-05-09,52.935714
3,ABW,2021-05-16,65.184000
4,AFG,2021-03-14,0.000000
...,...,...,...
1847,ZWE,2021-04-18,0.518571
1848,ZWE,2021-04-25,0.950000
1849,ZWE,2021-05-02,1.528571
1850,ZWE,2021-05-09,2.107143


In [92]:
df_merge = pd.merge(df_merge, vaccination_df, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,school_closures,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,0.000000,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.000000,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,1.285714,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,1.000000,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,0.950000
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,1.528571
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,1.000000,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,2.107143


### Youth Unemployment

Youth unemployment data for all countries, selecting last value occurence

In [93]:
youth_unemployment_df=read_csv_file('1Vv6uPieFDONx3kSA9V6vbgzJSireUJDN','youth_unemployment', 3)

#Select latest value between 2010 and 2020 
years=youth_unemployment_df.columns[54:-1]

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for index, row in youth_unemployment_df.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])
    
youth_unemployment_df['latest_value']=latest_values
youth_unemployment_df=youth_unemployment_df[['Country Code', 'latest_value']]
youth_unemployment_df.columns=['Code', '%youth_unemployment_total']
youth_unemployment_df

,Code,%youth_unemployment_total
0,ABW,NaN
1,AFG,17.219999
2,AGO,16.260000
3,ALB,26.990000
4,AND,NaN
...,...,...
259,XKX,NaN
260,YEM,24.240000
261,ZAF,57.470001
262,ZMB,22.629999


In [94]:
df_merge = pd.merge(df_merge, youth_unemployment_df, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,Holiday,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,0.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,1.0,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,0.0,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,0.950000,8.13
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,1.528571,8.13
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,0.0,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,2.107143,8.13


### Life Expectancy

Life expectancy values last occurence pero country and association in original dataframe with selected country 

In [95]:
life_expectancy_df=read_csv_file('1946XP73K-KBKOYjZMTrR-kObEkr7VZ23','life_expectancy', 3)

#Select latest value between 2010 and 2020 
years=life_expectancy_df.columns[54:-1]

#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for index, row in life_expectancy_df.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])
    
life_expectancy_df['latest_value']=latest_values
life_expectancy_df=life_expectancy_df[['Country Code', 'latest_value']]
life_expectancy_df.columns=['Code', 'life_expectancy']
life_expectancy_df

,Code,life_expectancy
0,ABW,76.152000
1,AFG,64.486000
2,AGO,60.782000
3,ALB,78.458000
4,AND,NaN
...,...,...
259,XKX,72.195122
260,YEM,66.096000
261,ZAF,63.857000
262,ZMB,63.510000


In [96]:
df_merge = pd.merge(df_merge, life_expectancy_df, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,temp,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,23.986111,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,0.950000,8.13,61.195
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,1.528571,8.13,61.195
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,NaN,NaN,NaN,1.7,2168000.0,32.237,2.107143,8.13,61.195


### % Of Population greater than 65

% Of Population greater than 65 values last occurence pero country and association in original dataframe with selected country 

In [97]:
df_population_gr_65=read_csv_file('1z4gqhcx55nLbhJ9v6__Re4UEglUt7-Dz','ages65_and_older', 3)
years=df_population_gr_65.columns[54:-1]

latest_values=[]

for index, row in df_population_gr_65.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])
    
df_population_gr_65['latest_value']=latest_values
df_population_gr_65=df_population_gr_65[['Country Code', 'latest_value']]
df_population_gr_65.columns=['Code', '%df_population_gr_65']
df_population_gr_65
df_merge = pd.merge(df_merge, df_population_gr_65, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,prcp,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,0.0,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,NaN,NaN,1.7,2168000.0,32.237,0.950000,8.13,61.195,2.980608
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,NaN,NaN,1.7,2168000.0,32.237,1.528571,8.13,61.195,2.980608
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,NaN,NaN,1.7,2168000.0,32.237,2.107143,8.13,61.195,2.980608


### Median Age

Median age values last occurence pero country and association in original dataframe with selected country 

In [98]:
df_median_age=read_csv_file('1jQ_mcxB7JKIlp5xJ9d4zSmjdJQyAwKp0','median-age',0)
years=df_median_age['Year'].unique()
years=years[1:15]
countries=df_median_age['Code'].unique()
countries = [x for x in countries if str(x) != 'nan']
latest_values=[]

for country in countries:
    max_year=2010
    for year in years:
        if((year > max_year) & (len(df_median_age[(df_median_age['Year']==year) & (df_median_age['Code']==country)]) > 0)):
            max_year = year

    latest_values.append(df_median_age[(df_median_age['Year']==max_year) & (df_median_age['Code']==country)]['UN Population Division (Median Age) (2017)'].values[0])
df_median_age=pd.DataFrame()
df_median_age['Code']=countries
df_median_age['UN Population Division (Median Age) (2017)']=latest_values
df_merge = pd.merge(df_merge, df_median_age, on=["Code"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,doctors_per_1000,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017)
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,NaN,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608,19.600000
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,NaN,1.7,2168000.0,32.237,0.950000,8.13,61.195,2.980608,19.600000
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,NaN,1.7,2168000.0,32.237,1.528571,8.13,61.195,2.980608,19.600000
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,NaN,1.7,2168000.0,32.237,2.107143,8.13,61.195,2.980608,19.600000


### Excess Mortality (Previous 7 days)

Excess Mortality associated with week n previous 7 days 

In [99]:
df_excess_mortality_prev=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-p-scores',0)
df_excess_mortality_prev['Date'] = pd.to_datetime(df_excess_mortality_prev['Day']) - pd.to_timedelta(7, unit='d')
df_excess_mortality_prev = df_excess_mortality_prev.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_excess_mortality_prev.columns=['Code', 'Date', 'excess_mortality_prev_7']
df_excess_mortality_prev

,Code,Date,excess_mortality_prev_7
0,ALB,2020-01-26,-10.65
1,ALB,2020-02-23,2.17
2,ALB,2020-03-29,0.62
3,ALB,2020-04-26,3.23
4,ALB,2020-05-24,6.15
...,...,...,...
3211,UZB,2020-11-29,10.71
3212,UZB,2020-12-27,6.41
3213,UZB,2021-01-24,17.12
3214,UZB,2021-02-21,4.76


In [100]:
df_merge = pd.merge(df_merge, df_excess_mortality_prev, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,nurses_per_1000,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017),excess_mortality_prev_7
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,NaN,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608,19.600000,NaN
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,NaN,1.7,2168000.0,32.237,0.950000,8.13,61.195,2.980608,19.600000,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,NaN,1.7,2168000.0,32.237,1.528571,8.13,61.195,2.980608,19.600000,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,NaN,1.7,2168000.0,32.237,2.107143,8.13,61.195,2.980608,19.600000,NaN


### Excess Mortality (18 days after) - Infectious value

Excess Mortality associated with week n (target week)

In [101]:
df_excess_mortality=read_csv_file('1tJBHRAMU_xPLljHaotn4rxeYP5JCkb9j','excess-mortality-p-scores',0)
df_excess_mortality['Date'] = pd.to_datetime(df_excess_mortality['Day'])
df_excess_mortality = df_excess_mortality.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')]).mean().reset_index()
df_excess_mortality.columns=['Code', 'Date', 'excess_mortality']
df_excess_mortality

,Code,Date,excess_mortality
0,ALB,2020-02-02,-10.65
1,ALB,2020-03-01,2.17
2,ALB,2020-04-05,0.62
3,ALB,2020-05-03,3.23
4,ALB,2020-05-31,6.15
...,...,...,...
3211,UZB,2020-12-06,10.71
3212,UZB,2021-01-03,6.41
3213,UZB,2021-01-31,17.12
3214,UZB,2021-02-28,4.76


In [102]:
df_merge = pd.merge(df_merge, df_excess_mortality, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,beds_per_1000,number_of_arrivals,urban_population,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017),excess_mortality_prev_7,excess_mortality
0,ABW,2020-03-08,0.0,-0.233333,-3.111000,1.911000,6.572333,9.755667,-8.889000,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
1,ABW,2020-03-15,0.0,1.051000,-3.050714,2.330000,8.013571,7.656429,-13.826429,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
2,ABW,2020-03-22,0.0,1.734714,0.755143,1.020286,3.979714,7.836857,-4.469286,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
3,ABW,2020-03-29,0.0,2.265429,3.775571,0.224571,3.775429,7.877714,2.020143,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
4,ABW,2020-04-05,0.0,-12.347000,5.428714,4.102143,-6.877714,0.489857,-3.795857,0.0,...,NaN,1102000.0,43.293,NaN,NaN,76.152,14.058355,41.200001,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7911,ZWE,2021-04-18,0.0,13.775571,39.591857,-0.469571,15.204286,17.244857,4.857143,0.0,...,1.7,2168000.0,32.237,0.518571,8.13,61.195,2.980608,19.600000,NaN,NaN
7912,ZWE,2021-04-25,0.0,12.591857,38.347000,0.857429,15.122286,20.796143,-10.183714,0.0,...,1.7,2168000.0,32.237,0.950000,8.13,61.195,2.980608,19.600000,NaN,NaN
7913,ZWE,2021-05-02,0.0,11.489714,34.816429,1.061286,15.040857,22.489857,-0.857000,0.0,...,1.7,2168000.0,32.237,1.528571,8.13,61.195,2.980608,19.600000,NaN,NaN
7914,ZWE,2021-05-09,0.0,13.999857,38.510429,0.918429,18.591714,23.530571,6.306143,0.0,...,1.7,2168000.0,32.237,2.107143,8.13,61.195,2.980608,19.600000,NaN,NaN


### Calculation of Raw Deaths

Since raw deaths are not available weekly we will calculated them based on averag deaths 2019-2015 and excess mortality previous n days, generating different values of deaths for target week and previous n days

In [103]:
df_raw_deaths=read_csv_file('1avaGZk0Zlrx786lM6wOdaF_74Ssq68K2','excess-mortality-raw-death-count',0)

df_raw_deaths=df_raw_deaths[['Code', 'Day', 'average_deaths_2015_2019_all_ages']]
df_raw_deaths['month']=pd.to_datetime(df_raw_deaths['Day']).dt.month
df_raw_deaths=df_raw_deaths[['Code', 'month', 'average_deaths_2015_2019_all_ages']]
df_raw_deaths=df_raw_deaths.groupby(['Code','month']).agg('sum').reset_index()
df_raw_deaths['average_deaths_2015_2019_all_ages']=df_raw_deaths['average_deaths_2015_2019_all_ages']/4

df_merge['month']=pd.to_datetime(df_merge['Date']).dt.month
df_merge = pd.merge(df_merge, df_raw_deaths, on=["Code", "month"])
df_merge['deaths_prev_7']=(df_merge['average_deaths_2015_2019_all_ages']/100)*df_merge['excess_mortality_prev_7']

df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,total_vaccinations_per_100,%youth_unemployment_total,life_expectancy,%df_population_gr_65,UN Population Division (Median Age) (2017),excess_mortality_prev_7,excess_mortality,month,average_deaths_2015_2019_all_ages,deaths_prev_7
0,AUS,2020-03-08,2.0,1.050000,1.722333,-0.739000,4.977667,-2.089000,7.927667,0.0,...,NaN,11.84,82.748780,15.921202,37.900002,4.62,4.67,3,3126.250,144.432750
1,AUS,2020-03-15,2.0,0.761714,1.510143,-1.286000,7.697286,-0.819857,10.567714,0.0,...,NaN,11.84,82.748780,15.921202,37.900002,7.28,4.62,3,3126.250,227.591000
2,AUS,2020-03-22,2.0,1.285714,3.367429,-1.081714,9.346857,0.938857,10.877857,0.0,...,NaN,11.84,82.748780,15.921202,37.900002,12.13,7.28,3,3126.250,379.214125
3,AUS,2020-03-29,2.0,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,0.0,...,NaN,11.84,82.748780,15.921202,37.900002,11.31,12.13,3,3126.250,353.578875
4,AUS,2021-03-07,2.0,-14.061000,-1.653143,5.306143,-37.918286,-16.959143,-10.775571,1.0,...,0.000000,11.84,82.748780,15.921202,37.900002,NaN,NaN,3,3126.250,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,USA,2021-01-31,1.0,-23.979714,-11.387857,11.449143,-39.571429,-22.591714,-28.999857,1.0,...,2.464000,8.30,78.539024,16.209606,38.299999,25.01,28.11,1,74249.625,18569.831213
3927,USA,2021-02-07,1.0,-24.673429,-12.857143,10.795714,-40.183714,-19.183714,-28.816286,1.0,...,4.160000,8.30,78.539024,16.209606,38.299999,15.60,25.01,2,57847.700,9024.241200
3928,USA,2021-02-14,1.0,-25.836714,-14.142714,10.734429,-41.285714,-21.000143,-28.816143,1.0,...,6.405714,8.30,78.539024,16.209606,38.299999,12.85,15.60,2,57847.700,7433.429450
3929,USA,2021-02-21,1.0,-25.938714,-14.551000,10.714000,-41.714286,-25.775714,-27.938714,1.0,...,9.125714,8.30,78.539024,16.209606,38.299999,8.74,12.85,2,57847.700,5055.888980


### Calculation of Accumulated Deaths 

For every country we start summing up number of detahs over time in order to retrieve accumulated deaths which will serve as index of population inmunity against the virus 

In [104]:
df_population=read_csv_file('14URaFs8YAJmJzXQuHzT7qV6uqGRUr9oW','populations',3)

In [105]:
#Select latest value between 2010 and 2020 
years=df_population.columns[54:-1]
years

Index(['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018',
       '2019', '2020'],
      dtype='object')

In [106]:
#Iterate through all values and collect the values for the maximum year recorded for the selected country
latest_values=[]

for index, row in df_population.iterrows():
    
    aux_year=2010
    
    for year in years:
        
        if((math.isnan(row[year])==False)):
            
            aux_year=year
            
    latest_values.append(row[str(aux_year)])

In [107]:
df_population['latest_value']=latest_values
df_population=df_population[['Country Code', 'latest_value']]
df_population.columns=['Code', 'population']
df_population

,Code,population
0,ABW,106314.0
1,AFG,38041754.0
2,AGO,31825295.0
3,ALB,2854191.0
4,AND,77142.0
...,...,...
259,XKX,1794248.0
260,YEM,29161922.0
261,ZAF,58558270.0
262,ZMB,17861030.0


In [108]:
df_merge = pd.merge(df_merge, df_population, on=["Code"], how="left")
df_merge=df_merge.sort_values(['Code','Date'], ascending=[True, True])

In [109]:
df_merge=df_merge.sort_values(['Code','Date'], ascending=[True, True])

In [110]:
accumulated=[]
country='AUS'
cumsum=0

for index, row in df_merge.iterrows():
    country_last=row['Code']
    
    if(country!=country_last):
        
        cumsum=0
        country=row['Code']
        if(row['deaths_prev_7']  > 0):
            cumsum=cumsum+row['deaths_prev_7'] 
        
    else:
        
        if(row['deaths_prev_7']  > 0):
            cumsum=cumsum+row['deaths_prev_7'] 
    
    accumulated.append(cumsum)

df_merge['accumulated']=accumulated
df_merge['accumulated']=100*df_merge['accumulated']/df_merge['population']

## Reproduction Rate

In [111]:
df_reproduction_rate=read_csv_file('1t4TSf6qMxrAlgy1dqyMcml2Mum1klLiS','owid-covid-data',0)
df_reproduction_rate=df_reproduction_rate[['iso_code', 'date', 'reproduction_rate']]
df_reproduction_rate.columns=['Code', 'Date', 'reproduction_rate']
df_reproduction_rate['Date'] = pd.to_datetime(df_reproduction_rate['Date'])+ pd.to_timedelta(18, unit='d')
df_reproduction_rate = df_reproduction_rate.groupby(['Code', pd.Grouper(key='Date', freq='W-SUN')])\
                                           .mean().reset_index()

In [112]:
df_merge = pd.merge(df_merge, df_reproduction_rate, on=["Code", "Date"], how="left")
df_merge

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,%df_population_gr_65,UN Population Division (Median Age) (2017),excess_mortality_prev_7,excess_mortality,month,average_deaths_2015_2019_all_ages,deaths_prev_7,population,accumulated,reproduction_rate
0,AUS,2020-03-08,2.0,1.050000,1.722333,-0.739000,4.977667,-2.089000,7.927667,0.0,...,15.921202,37.900002,4.62,4.67,3,3126.25,144.432750,25364307.0,0.000569,NaN
1,AUS,2020-03-15,2.0,0.761714,1.510143,-1.286000,7.697286,-0.819857,10.567714,0.0,...,15.921202,37.900002,7.28,4.62,3,3126.25,227.591000,25364307.0,0.001467,NaN
2,AUS,2020-03-22,2.0,1.285714,3.367429,-1.081714,9.346857,0.938857,10.877857,0.0,...,15.921202,37.900002,12.13,7.28,3,3126.25,379.214125,25364307.0,0.002962,NaN
3,AUS,2020-03-29,2.0,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,0.0,...,15.921202,37.900002,11.31,12.13,3,3126.25,353.578875,25364307.0,0.004356,2.170000
4,AUS,2020-04-05,2.0,-0.898000,15.163286,0.448857,-1.061286,-4.959143,6.959000,0.0,...,15.921202,37.900002,6.36,11.31,4,2564.75,163.118100,25364307.0,0.004999,2.330000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,USA,2021-04-11,1.0,-8.816429,-4.244714,7.040714,-27.183571,4.265286,-25.775714,1.0,...,16.209606,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.021429
3927,USA,2021-04-18,1.0,-8.469429,-4.000143,6.632286,-26.163143,12.040857,-24.877429,1.0,...,16.209606,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.074286
3928,USA,2021-04-25,1.0,-8.020429,-0.142857,6.449143,-25.469286,17.755000,-27.979714,1.0,...,16.209606,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.047143
3929,USA,2021-05-02,1.0,-7.979571,-1.816286,5.816143,-24.571429,24.347000,-26.428571,1.0,...,16.209606,38.299999,NaN,NaN,5,64689.30,NaN,328239523.0,0.234450,1.042857


In [113]:
df_final=df_merge.dropna(subset=['reproduction_rate'])

In [114]:
df_final

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,%df_population_gr_65,UN Population Division (Median Age) (2017),excess_mortality_prev_7,excess_mortality,month,average_deaths_2015_2019_all_ages,deaths_prev_7,population,accumulated,reproduction_rate
3,AUS,2020-03-29,2.0,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,0.000000,...,15.921202,37.900002,11.31,12.13,3,3126.25,353.578875,25364307.0,0.004356,2.170000
4,AUS,2020-04-05,2.0,-0.898000,15.163286,0.448857,-1.061286,-4.959143,6.959000,0.000000,...,15.921202,37.900002,6.36,11.31,4,2564.75,163.118100,25364307.0,0.004999,2.330000
5,AUS,2020-04-12,2.0,-10.591857,22.020286,5.102143,-21.836714,-14.836714,-0.918286,0.285714,...,15.921202,37.900002,2.95,6.36,4,2564.75,75.660125,25364307.0,0.005297,2.280000
6,AUS,2020-04-19,2.0,-32.163429,-0.020429,13.428429,-48.775571,-28.183714,-22.816286,2.000000,...,15.921202,37.900002,5.75,2.95,4,2564.75,147.473125,25364307.0,0.005879,1.457143
7,AUS,2020-04-26,2.0,-40.918429,-12.347143,17.612143,-60.204143,-39.326571,-35.265286,2.000000,...,15.921202,37.900002,3.18,5.75,4,2564.75,81.559050,25364307.0,0.006200,0.765714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,USA,2021-04-11,1.0,-8.816429,-4.244714,7.040714,-27.183571,4.265286,-25.775714,1.000000,...,16.209606,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.021429
3927,USA,2021-04-18,1.0,-8.469429,-4.000143,6.632286,-26.163143,12.040857,-24.877429,1.000000,...,16.209606,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.074286
3928,USA,2021-04-25,1.0,-8.020429,-0.142857,6.449143,-25.469286,17.755000,-27.979714,1.000000,...,16.209606,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.047143
3929,USA,2021-05-02,1.0,-7.979571,-1.816286,5.816143,-24.571429,24.347000,-26.428571,1.000000,...,16.209606,38.299999,NaN,NaN,5,64689.30,NaN,328239523.0,0.234450,1.042857


In [173]:
rate_tendency=[]
country='AUS'

for index, row in df_final.iterrows():
    country_last=row['Code']
    
    if(country==country_last):
            
        date_1 = pd.to_datetime(row['Date']) - pd.to_timedelta(7, unit='d')
        date_2 = pd.to_datetime(row['Date']) - pd.to_timedelta(14, unit='d')
    
        s_1 = df_final[(df_final['Code']==country) & (pd.to_datetime(df_final['Date'])== date_1)]['reproduction_rate']
        s_2 = df_final[(df_final['Code']==country) & (pd.to_datetime(df_final['Date'])== date_2)]['reproduction_rate']
        
        if (s_1.size==0 & s_2.size==0):
            
            n=row['reproduction_rate']
        
        else: 
            if (s_2.size==0):
            
                n=s_1.iloc[0]
        
            else: 
                n=s_1.iloc[0]/s_2.iloc[0]
            
        print(n)
        rate_tendency.append(n)
    
    else:
        country=country_last
        rate_tendency.append(row['reproduction_rate'])

rate_tendency

2.17
2.17
1.0737327188940091
0.9785407725321891
0.6390977443609022
0.5254901960784315
0.6119402985074627
0.8475609756097561
1.158273381294964
1.5621118012422364
1.0815109343936384
0.9944852941176467
1.1109057301293899
1.1064891846921798
0.9654135338345865
1.524922118380062
1.1624106230847806
1.181019332161687
0.9709821428571428
0.8459770114942528
0.8849637681159422
0.9160696008188329
0.8826815642458101
0.779746835443038
0.8409090909090909
0.862934362934363
0.9597315436241611
0.979020979020979
0.961904761904762
0.9504950495049505
1.119791666666667
1.3325581395348833
1.18673647469459
0.9455882352941175
0.9580093312597202
0.883116883116883
1.0919117647058825
1.531986531986532
0.7901098901098901
0.9262865090403338
1.027027027027027
1.2236842105263162
1.1839904420549578
0.7921291624621596
0.8687898089171973
0.8724340175953078
0.8924369747899161
0.8832391713747647
1.1556503198294241
1.1457564575645758
0.996779388083736
1.176090468497577
1.2266483516483513
0.9932810750279958
0.873731679819616

[2.17,
 2.17,
 1.0737327188940091,
 0.9785407725321891,
 0.6390977443609022,
 0.5254901960784315,
 0.6119402985074627,
 0.8475609756097561,
 1.158273381294964,
 1.5621118012422364,
 1.0815109343936384,
 0.9944852941176467,
 1.1109057301293899,
 1.1064891846921798,
 0.9654135338345865,
 1.524922118380062,
 1.1624106230847806,
 1.181019332161687,
 0.9709821428571428,
 0.8459770114942528,
 0.8849637681159422,
 0.9160696008188329,
 0.8826815642458101,
 0.779746835443038,
 0.8409090909090909,
 0.862934362934363,
 0.9597315436241611,
 0.979020979020979,
 0.961904761904762,
 0.9504950495049505,
 1.119791666666667,
 1.3325581395348833,
 1.18673647469459,
 0.9455882352941175,
 0.9580093312597202,
 0.883116883116883,
 1.0919117647058825,
 1.531986531986532,
 0.7901098901098901,
 0.9262865090403338,
 1.027027027027027,
 1.2236842105263162,
 1.1839904420549578,
 0.7921291624621596,
 0.8687898089171973,
 0.8724340175953078,
 0.8924369747899161,
 0.8832391713747647,
 1.1556503198294241,
 1.145756457

In [174]:
len(rate_tendency)

3586

In [175]:
df_final['rate_tendency']=rate_tendency

<ipython-input-175-86b43dc90934>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['rate_tendency']=rate_tendency


In [176]:
df_final

,Code,Date,contact_tracing,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces,debt_relief,...,UN Population Division (Median Age) (2017),excess_mortality_prev_7,excess_mortality,month,average_deaths_2015_2019_all_ages,deaths_prev_7,population,accumulated,reproduction_rate,rate_tendency
3,AUS,2020-03-29,2.0,0.714429,8.979571,-0.387571,5.448857,-2.857143,8.326571,0.000000,...,37.900002,11.31,12.13,3,3126.25,353.578875,25364307.0,0.004356,2.170000,2.170000
4,AUS,2020-04-05,2.0,-0.898000,15.163286,0.448857,-1.061286,-4.959143,6.959000,0.000000,...,37.900002,6.36,11.31,4,2564.75,163.118100,25364307.0,0.004999,2.330000,2.170000
5,AUS,2020-04-12,2.0,-10.591857,22.020286,5.102143,-21.836714,-14.836714,-0.918286,0.285714,...,37.900002,2.95,6.36,4,2564.75,75.660125,25364307.0,0.005297,2.280000,1.073733
6,AUS,2020-04-19,2.0,-32.163429,-0.020429,13.428429,-48.775571,-28.183714,-22.816286,2.000000,...,37.900002,5.75,2.95,4,2564.75,147.473125,25364307.0,0.005879,1.457143,0.978541
7,AUS,2020-04-26,2.0,-40.918429,-12.347143,17.612143,-60.204143,-39.326571,-35.265286,2.000000,...,37.900002,3.18,5.75,4,2564.75,81.559050,25364307.0,0.006200,0.765714,0.639098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3926,USA,2021-04-11,1.0,-8.816429,-4.244714,7.040714,-27.183571,4.265286,-25.775714,1.000000,...,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.021429,1.061192
3927,USA,2021-04-18,1.0,-8.469429,-4.000143,6.632286,-26.163143,12.040857,-24.877429,1.000000,...,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.074286,1.084977
3928,USA,2021-04-25,1.0,-8.020429,-0.142857,6.449143,-25.469286,17.755000,-27.979714,1.000000,...,38.299999,NaN,NaN,4,53949.95,NaN,328239523.0,0.234450,1.047143,1.051748
3929,USA,2021-05-02,1.0,-7.979571,-1.816286,5.816143,-24.571429,24.347000,-26.428571,1.000000,...,38.299999,NaN,NaN,5,64689.30,NaN,328239523.0,0.234450,1.042857,0.974734


In [177]:
df_final.to_csv('../df_covid_processed_FINAL_new_r0_chile.csv')